In [1]:
from msu_esd import Pipe, hardy_cross
import numpy as np

\pagebreak

# Example 1.11
![Network Schematic](figures/hardy_example.png)

| Pipe | L (ft) | D (in) | K   | C   | $\epsilon$ (ft) |
|------|--------|--------|-----|-----|-----------------|
| 1    | 2000   | 12     | 0   | 0   | 0.00015         |
| 2    | 2000   | 8      | 0   | 0   | 0.00015         |
| 3    | 3000   | 6      | 0   | 0   | 0.00015         |
| 4    | 4000   | 6      | 0   | 0   | 0.00015         |
| 5    | 1000   | 8      | 0   | 0   | 0.00015         |
| 6    | 3000   | 8      | 0   | 0   | 0.00015         |
| 7    | 2000   | 8      | 0   | 0   | 0.00015         |

The guess values are,

![Guess Values](figures/hardy_guess.png)

Define pipe objects first.

In [2]:
epsilon = 0.00015
rho = 1.94
mu = 3.104e-5

pipe1 = Pipe(1, 2000, epsilon, rho, mu)
pipe2 = Pipe(8/12, 2000, epsilon, rho, mu)
pipe3 = Pipe(6/12, 3000, epsilon, rho, mu)
pipe4 = Pipe(6/12, 4000, epsilon, rho, mu)
pipe5 = Pipe(8/12, 1000, epsilon, rho, mu)
pipe6 = Pipe(8/12, 3000, epsilon, rho, mu)
pipe7 = Pipe(8/12, 2000, epsilon, rho, mu)

Define the connection matrix and flow rates like so,

In [3]:
Q_guess = np.array([0.8, 0.2, 1.2, 1.2, 1, 1, 1])
N = np.transpose([
    [1, -1, -1, -1, 0, 0, 0],
    [-1, 0, 0, 0, 1, 1, -1]
])
N

array([[ 1, -1],
       [-1,  0],
       [-1,  0],
       [-1,  0],
       [ 0,  1],
       [ 0,  1],
       [ 0, -1]])

## No Additional Devices
Utilize the hardy cross function.

In [4]:
Q = hardy_cross([pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7],
                Q_guess, N)
Q

array([ 1.8661928 , -0.76214598,  0.23785402,  0.23785402,  0.89595318,
        0.89595318,  1.10404682])

## Heat Exchanger in Line 1
A list of functions needs to be inputting in for `h` and `dh`. If the device loss has a value of $50Q_1\,^2$,

In [5]:
h = [
    lambda Q_: 50*Q_*abs(Q_),  # Positive because loss is considered positive (e.g. turbine)
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0
]

dh = [
    lambda Q_: 100*abs(Q_),
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0,
    lambda Q_: 0
]

Q = hardy_cross([pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7],
                Q_guess, N, h=h, dh=dh)
Q

array([ 0.80979017, -0.43632714,  0.56367286,  0.56367286,  1.62653696,
        1.62653696,  0.37346304])

## Adding a Pump to Line 1
If we add a pump with a head of 203.5 ft,

In [6]:
# Only need to change the function in the first index
h[0] = lambda Q_: 50*Q_*abs(Q_) - 203.5  # Subtract because pump means the system is doing work
Q = hardy_cross([pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7],
                Q_guess, N, h=h, dh=dh)
Q

array([ 2.0000145 , -0.81268625,  0.18731375,  0.18731375,  0.81267175,
        0.81267175,  1.18732825])